In [5]:
# setting the environment variables
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

from config import set_environment
set_environment()

In [1]:
from huggingface_hub import list_models

def list_most_popular(task: str):
    for rank, model in enumerate(
        list_models(filter=task, sort="downloads", direction=-1)
):
        if rank == 5:
            break
        print(f"{model.id}, {model.downloads}\n")

list_most_popular("text-classification")

1231czx/llama3_it_ultra_list_and_bold500, 38633004

distilbert/distilbert-base-uncased-finetuned-sst-2-english, 6346490

cross-encoder/ms-marco-MiniLM-L-4-v2, 5228573

cardiffnlp/twitter-roberta-base-sentiment, 4087943

facebook/bart-large-mnli, 2813035



In [2]:
list_most_popular("summarization")

google-t5/t5-small, 8393079

facebook/bart-large-cnn, 3872105

google-t5/t5-base, 2335764

sshleifer/distilbart-cnn-12-6, 679203

google-t5/t5-large, 614868



In [13]:
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint

customer_email = """
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spirit but also any semblance of confidence I had placed in your esteemed brand.

Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine. Its once elegant exterior was marred by the scars of travel, resembling a war-torn soldier who had fought valiantly on the fields of some espresso battlefield. This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable
"""  # created by GPT-3.5

# Please make sure you've set the environment variable HUGGINGFACEHUB_API_TOKEN with your API token
summarizer = HuggingFaceEndpoint(
    repo_id="facebook/bart-large-cnn", temperature=0,
    model_kwargs={"max_length":180}
)
def summarize(llm, text) -> str:
    return llm(f"Summarize this: {text}!")



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/alois/.cache/huggingface/token
Login successful


In [14]:
summarize(summarizer, customer_email)

BadRequestError:  (Request ID: CKAEU3zbVEKngMRY_6NWk)

Bad request:
The following `model_kwargs` are not used by the model: ['watermark', 'stop_sequences', 'stop', 'return_full_text'] (note: typos in the generate arguments will also show up in this list)

In [8]:
from transformers import pipeline

sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    device=0
)

/home/alois/Dokumente/development/github/generative_ai_with_langchain/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
print(sentiment_model(customer_email))

[{'label': 'LABEL_0', 'score': 0.5822023153305054}]


In [10]:
print(sentiment_model("I am so angry and sad, I want to kill myself!"))

[{'label': 'LABEL_0', 'score': 0.9788626432418823}]


In [11]:
print(sentiment_model("I am elated, I am so happy, this is the best thing that ever happened to me!"))

[{'label': 'LABEL_2', 'score': 0.9926880598068237}]


In [12]:
print(sentiment_model("I don't care. I guess it's ok, or not, I couldn't care one way or the other"))


[{'label': 'LABEL_1', 'score': 0.5958544015884399}]


The code provided in the next cell does not work because a Google Cloud project has not been created yet. When working with Google Cloud services, it is essential to have a project set up in the Google Cloud Console. This project acts as a container for all your Google Cloud resources, including APIs, billing, and permissions.

#### Why the Code Fails
1. **Missing Project ID**: Google Cloud services require a project ID to identify which project the resources and APIs belong to.
2. **Authentication**: Without a project, you cannot authenticate your requests to Google Cloud services.
3. **APIs and Services**: Specific APIs and services need to be enabled within a Google Cloud project to be accessible.

#### Steps to Resolve the Issue
1. **Create a Google Cloud Project**:
   - Go to the [Google Cloud Console](https://console.cloud.google.com/).
   - Click on the project drop-down and select "New Project".
   - Enter a project name and click "Create".

2. **Enable Required APIs**:
   - Navigate to the "APIs & Services" section in the Google Cloud Console.
   - Enable the APIs that your code will use (e.g., Google Cloud Storage, Google Cloud Pub/Sub).

3. **Set Up Authentication**:
   - Go to the "IAM & Admin" section and create a service account.
   - Download the service account key file (JSON) and set the `GOOGLE_APPLICATION_CREDENTIALS` environment variable to point to this file.

4. **Update Your Code**:
   - Ensure your code includes the project ID and uses the correct authentication credentials.

By following these steps, you will set up a Google Cloud project and enable your code to interact with Google Cloud services successfully.

In [15]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: {email}
Category:
"""
prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(llm_chain.run(customer_email))

/home/alois/Dokumente/development/github/generative_ai_with_langchain/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(
/home/alois/Dokumente/development/github/generative_ai_with_langchain/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warn

ValidationError: 1 validation error for VertexAI
__root__
  Unable to find your project. Please provide a project ID by:
- Passing a constructor argument
- Using vertexai.init()
- Setting project using 'gcloud config set project my-project'
- Setting a GCP environment variable
- To create a Google Cloud project, please follow guidance at https://developers.google.com/workspace/guides/create-project (type=value_error)